<a href="https://colab.research.google.com/github/wasif31/Artificial-Intelligence-Course-Lab/blob/main/5.Simple_Text_Suggesting_System_Using_Naive_Bayes/Simple_text_suggesting_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk import trigrams 
from nltk.corpus import reuters
from collections import  defaultdict
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus

In [43]:
wiki = WikiCorpus(datapath('bgwiki-latest-pages-articles-shortened.xml.bz2'), lemmatize=False).get_texts()

In [27]:
print(wiki)

<generator object WikiCorpus.get_texts at 0x7f2a22f5cf68>


In [28]:
# get Reuters DataSet
nltk.download('punkt')
nltk.download('reuters')
rSentences  = reuters.sents()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [29]:
print(rSentences)

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]


In [30]:
def measureProbability(sentenceModel):
  for nextWord in sentenceModel:
    nextWords = sentenceModel[nextWord]
    total_Word_Count = float(sum(nextWords.values()))
    for previousWord in nextWords:
      nextWords[previousWord]/=total_Word_Count

In [31]:
def measureSigleWordProbability(sentenceModel,wordCount):
  for word in sentenceModel:
    sentenceModel[word]/=wordCount

In [32]:
def convertToLower(item):
  if type(item)==str:
    return item.lower()
  return item

In [38]:
sentenceModelLambda1 = defaultdict(lambda: set())
sentenceModelLambda2 = defaultdict(lambda: set())

In [39]:
def measureWordCount(sentenceModel1,sentenceModel2,sentenceModel3,sentences):
  wordCount = 0
  for sentence in sentences:
    for word in sentence:
      wordCount+=1
      sentenceModel1[word]+=1
    for previousWord2,previousWord1,nextWord in trigrams(sentence,pad_right=True,pad_left=True):
      previousWord1 = convertToLower(previousWord1)
      previousWord2 = convertToLower(previousWord2)
      nextWord = convertToLower(nextWord)
      sentenceModel2[nextWord][previousWord1]+=1
      sentenceModel3[nextWord][previousWord2]+=1
      sentenceModelLambda1[previousWord1].add(nextWord)
      sentenceModelLambda2[previousWord2].add(nextWord)

  return wordCount

In [40]:
sentenceModel1 = defaultdict(lambda:0)
sentenceModel2 = defaultdict(lambda: defaultdict(lambda:0))
sentenceModel3 = defaultdict(lambda: defaultdict(lambda:0))

In [44]:
wikiCount = measureWordCount(sentenceModel1,sentenceModel2,sentenceModel3,wiki)
print(wikiCount)

1321


In [46]:
reutersCount = measureWordCount(sentenceModel1,sentenceModel2,sentenceModel3,rSentences)
print(reutersCount)

1720917


In [48]:
measureProbability(sentenceModel2)
measureProbability(sentenceModel3)

In [50]:
total_word = wikiCount + reutersCount
measureSigleWordProbability(sentenceModel1,total_word)

In [52]:
#Naive Bais
maxProbabilityWords = []
def makeWordSuggestionByTrigram(previousWord2,previousWord1):
  for nextWord in sentenceModelLambda1[previousWord1] & sentenceModelLambda2[previousWord2]:
    naiveBiasTrigramWeight = sentenceModel1[nextWord]*sentenceModel2[nextWord][previousWord1]*sentenceModel3[nextWord][previousWord2]
    maxProbabilityWords.append((nextWord,naiveBiasTrigramWeight))
makeWordSuggestionByTrigram('my','name')
maxProbabilityWords.sort(key=lambda o:o[1],reverse=True)
print(*maxProbabilityWords[:10])   

('is', 3.431547440288585e-08) ('to', 1.3752228741704169e-09) (',', 7.061402167696784e-10) ('in', 5.9269638955901e-10) ('and', 5.794755236662483e-10) ('for', 4.7519342512442e-10) ('would', 4.4905347502451374e-10) ('will', 3.633936425372419e-10) ('that', 2.6531286744053744e-10) ('or', 2.1882008593022177e-10)


In [ ]:
while(True):
    text = input("Enter your line: ")
    if text == "stop":
        print("Ending The Program.....")
        break
    
    else:
        try:
            maxProbabilityWords = []
            text = text.split(" ")
            makeWordSuggestionByTrigram(text[0],text[1])
            maxProbabilityWords.sort(key=lambda o:o[1],reverse=True)
            print(*maxProbabilityWords[:10])
            
        except:
            continue

Enter your line: i eat
('you', 1.8658651527582323e-08) ("'", 5.356741200704024e-09) ('the', 6.725528736637289e-10) ('about', 3.4155520602873625e-10) ('their', 2.338459382026355e-10) ('.', 6.131166297584641e-12)
Enter your line: i will go
('not', 1.263555582751872e-06) ('be', 7.274640593245656e-07) ('have', 1.5586237336878607e-07) ('propose', 9.956101541154977e-08) ('see', 7.802458371769073e-08) ('assume', 7.741864558402111e-08) ('tell', 6.751626068373934e-08) ('give', 6.60811505542275e-08) ('take', 6.066780585490053e-08) ('admit', 5.806398418801583e-08)
Enter your line: how are 
('rare', 8.572994961237598e-08) ('used', 3.7867228822474515e-08) ('set', 3.6575907982268033e-08) ('the', 3.091705179842051e-08) ('subject', 2.9501174832726934e-08) ('interpreted', 1.75799260084823e-08) ('either', 1.5109379698856793e-08) ('you', 1.4926921222065857e-08) ('non', 1.4461913477464954e-08) ('better', 1.3816505071707667e-08)
Enter your line: who are
('not', 2.014393342831373e-06) ('terrified', 5.806398